In [2]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [3]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)

In [4]:
wiki

WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/home/ppk/Documents/LLM_Project/env/lib/python3.11/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200))

In [5]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)

vectordb = FAISS.from_documents(documents,OllamaEmbeddings())
retriever = vectordb.as_retriever()

USER_AGENT environment variable not set, consider setting it to identify your requests.
/tmp/ipykernel_158254/300577962.py:10: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  vectordb = FAISS.from_documents(documents,OllamaEmbeddings())


In [6]:
from langchain.tools.retriever import create_retriever_tool

langsmith_search = create_retriever_tool(retriever, "langsmith_search", "Search for information about langsmith. " \
"For any question about langsmith, you must use this tool")

In [7]:
langsmith_search.name

'langsmith_search'

In [8]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [9]:
tools = [wiki, arxiv, langsmith_search]

In [10]:
from langchain_ollama import OllamaLLM
#from langchain_community.llms import Ollama
llm = OllamaLLM(model="llama2")


In [11]:
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder, PromptTemplate,ChatPromptTemplate

# Define the system prompt (helpful assistant message)
system_prompt = SystemMessagePromptTemplate.from_template("You are a helpful assistant")

# Define the human message prompt
human_prompt = HumanMessagePromptTemplate.from_template("{input}")

# Define placeholders for chat history and agent scratchpad
chat_history_placeholder = MessagesPlaceholder(variable_name="chat_history", optional=True)
agent_scratchpad_placeholder = MessagesPlaceholder(variable_name="agent_scratchpad")

# Combine them into a single chat prompt template
# chat_prompt = ChatPromptTemplate.from_messages([
#     ("system", system_prompt),
#     ("human", human_prompt),
#     ("chat_history", chat_history_placeholder),
#     ("agent_scratchpad", agent_scratchpad_placeholder)
# ])
chat_prompt = ChatPromptTemplate([system_prompt,human_prompt,chat_history_placeholder,agent_scratchpad_placeholder])


In [ ]:
from langchain.agents import initialize_agent
from langchain.agents import AgentType

agent = initialize_agent(tools=tools, llm=llm,agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION, prompt_template=chat_prompt,
 verbose=True)
agent

AgentExecutor(verbose=True, tags=['zero-shot-react-description'], agent=ZeroShotAgent(llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['', 'agent_scratchpad', 'input', 'page_content'], input_types={}, partial_variables={}, template='Answer the following questions as best you can. You have access to the following tools:\n\nwikipedia - A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.\narxiv - A wrapper around Arxiv.org Useful for when you need to answer questions about Physics, Mathematics, Computer Science, Quantitative Biology, Quantitative Finance, Statistics, Electrical Engineering, and Economics from scientific articles on arxiv.org. Input should be a search query.\nlangsmith_search(query: \'str\', *, retriever: \'BaseRetriever\' = VectorStoreRetriever(tags=[\'FAISS\', \'OllamaEmbeddings\'], vectorstore=<langchain_commu

In [18]:
from langchain.agents import AgentExecutor

agent_executer = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executer

AgentExecutor(verbose=True, agent=RunnableAgent(runnable=AgentExecutor(verbose=True, tags=['zero-shot-react-description'], agent=ZeroShotAgent(llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['', 'agent_scratchpad', 'input', 'page_content'], input_types={}, partial_variables={}, template='Answer the following questions as best you can. You have access to the following tools:\n\nwikipedia - A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.\narxiv - A wrapper around Arxiv.org Useful for when you need to answer questions about Physics, Mathematics, Computer Science, Quantitative Biology, Quantitative Finance, Statistics, Electrical Engineering, and Economics from scientific articles on arxiv.org. Input should be a search query.\nlangsmith_search(query: \'str\', *, retriever: \'BaseRetriever\' = VectorStoreRetriever(tags=[\'FAI

In [21]:
agent.invoke({"input":"Tell me about langsmith"})




> Entering new AgentExecutor chain...


ValueError: Missing some input keys: {'', 'page_content'}

In [20]:
from langchain.agents import create_react_agent
agent2 = create_react_agent(llm=llm,tools=tools,prompt=chat_prompt)

ValueError: Prompt missing required variables: {'tools', 'tool_names'}